In [1]:
import pandas as pd
import tensorflow as tf
import requests
import json
import base64

In [15]:
data = pd.read_csv('data/stroke.csv')

ROW_TESTING = 1 

STROKE_LABELS = {
    0: "Normal",
    1: "stroke"
}

In [16]:
data_test = data.iloc[ROW_TESTING, :12].to_dict()
data_test

{'gender': 'Female',
 'age': 13,
 'hypertension': 0,
 'heart_disease': 0,
 'ever_married': 'No',
 'work_type': 'children',
 'Residence_type': 'Rural',
 'avg_glucose_level': 85.81,
 'bmi': 18.6,
 'smoking_status': 'Unknown',
 'stroke': 0}

In [17]:
stroke_value = data_test["stroke"]
stroke_label = STROKE_LABELS.get(stroke_value, "Unknown")
print(stroke_label)

Normal


In [18]:
data_test = data.iloc[ROW_TESTING, :10].to_dict()
data_test

{'gender': 'Female',
 'age': 13,
 'hypertension': 0,
 'heart_disease': 0,
 'ever_married': 'No',
 'work_type': 'children',
 'Residence_type': 'Rural',
 'avg_glucose_level': 85.81,
 'bmi': 18.6,
 'smoking_status': 'Unknown'}

In [19]:
def prepare_feature_spec(test_input: dict):
    feature_spec = {}
    for key, value in test_input.items():
        if isinstance(value, float):
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))
    return feature_spec

def serialize_example(feature_spec):
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec))
    return example.SerializeToString()

def encode_example(example):
    return base64.b64encode(example).decode()

def prepare_data(test_input):
    feature_spec = prepare_feature_spec(test_input)
    serialized_example = serialize_example(feature_spec)
    encoded_example = encode_example(serialized_example)
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [{
            "examples": {"b64": encoded_example}
        }]
    })

In [20]:
json_data = prepare_data(data_test)

endpoint = "http://103.190.215.127:8501/v1/models/cc-model:predict"
response = requests.post(endpoint, data=json_data)

predictions = response.json().get("predictions")
result = "Normal" if predictions and predictions[0][0] < 0.5 else "stroke" if predictions else "Error: No predictions found."

print("Prediction Result: " + result)
print("   Actually Label: " + stroke_label)

Prediction Result: Normal
   Actually Label: Normal
